>**AutoGen > Use Cases > Multi-agent Conversation Framework**
https://microsoft.github.io/autogen/0.2/docs/Use-Cases/agent_chat

# Multi-agent Conversation Framework

AutoGen은 파운데이션 모델 사용을 위한 **고수준 추상화**로서 통합된 **다중 에이전트 대화 프레임워크**를 제공한다. 이 프레임워크는 `LLM`, `도구(Tools)`, `자동화된 에이전트 대화`가 통합된 강력하고 사용자 정의가 가능한 **Conversable(대화 가능)** 에이전트를 특징으로 한다. 여러 유능한 에이전트 간의 대화를 자동화함으로써, **도구 사용을 포함한 작업**을 **인간 피드백이 있거나 없는 상황에서도 자율적으로 수행하도록 쉽게 구성**할 수 있다. 

이 프레임 워크는 복잡한 LLM 워크플로우의 `orchestration(조율)`, `automation(자동화)`, `optimization(최적화)`를 단순화한다. 또한 LLM 모델의 성능을 극대화하고 기존의 한계를 극복할 수 있게 해준다.
이를 통해, **다중 에이전트 대화**를 기반으로 한 차세대 LLM 애플리케이션을 최소한의 노력으로 구축할 수 있다.

### Agents

AutoGen은 **대화 기반**으로 과제를 해결하는 **conversable agent**를 추상화하고 구현한다.
특히 AutoGen의 에이전트들은 다음과 같은 특성을 갖는다.
- `Conversable` (대화 가능) : AutoGen의 에이전트는 서로 메시지를 주고받을 수 있다. 즉, **어떤 에이전트든 다른 에이전트와의 대화를 시작하거나 이어갈 수 있는 능력**을 가진다.
- `Customizable` (사용자 정의 가능) : AutoGen의 에이전트는 *LLM*, *인간*, *도구(Tools)*, 또는 이들의 조합을 통합하도록 유연하게 커스터마이징할 수 있다.

아래 그림은 AutoGen에 내장된 에이전트들을 보여준다.
![](https://velog.velcdn.com/images/heyggun/post/67f89ad9-52ed-44c1-a947-05d242efd1ed/image.png)

이 구조는 하나의 메인 에이전트가 복잡한 질의에 대해 내부적으로 **다중 에이전트**와 협업하여 **하위 질의 처리**를 수행하고 있다. AutoGen 프레임워크 기반의 에이전트 구성도로 `ConversableAgent`를 기반으로 다양한 타입의 하위 에이전트들이 어떻게 정의되고 연결되는지 보여준다.

핵심 구성요소는 **ConversableAgent**로, 이 모든 에이전트들은 공통적으로 `ConversableAgent` 클래스르 기반으로 확장된다. `human_input_mode`, `code_execute_config`, `DEFAULT_SYSTEM_MESSAGE` 등의 설정이 가능하다. 

하위 에이전트 타입으로 `AssisantAgent`가 있어 AI 어시스턴트 역할을 수행하여 `human_input_mode="NEVER", "code_execution_config=False"로 되어 있어, 사람의 직접 입력 없이 완전히 자동화되고 코드 실행 기능이 비활성화된 LLM 만을 사용해서 답변하는 에이전트 이다.

`UserProxyAgent`는 실제 사용자와 인터페이스하고 `human_input_mode="ALWAYS"로 되어 있어 사용자 입력을 항상 요청한다. 코드 실행 환경 및 사용자 피드백을 통해 상호작용하는 에이전트이다.

`GroupChatManager`는 여러 에이전트를 묶어 그룹 대화를 수행하고, `human_input_mode="NEVER"`로 자동화 되어 있다. 현재 위 그림에서는 `group_chat=[🟦, 🟩, 🟦, 🟩]`로 되어있는데 AssistantAgent 2개 + UserProxyAgent가 2개로 구성되어 있는 것을 볼 수 있다. LLM 에이전트들 간의 협업 과정 조율과 중재를 한다.

모두 `ConversableAgent`의 서브 클래스로 각 에이전트는 독립적인 역할을 갖고 있다. 여기서 GroupChatManager가 복수의 에이전트를 조합해 대화의 흐름을 만들 수 있는 구조라고 이해하면 된다.


---


Autogen에서는  `ConversableAgent` 라는 설계된 범용 클래스가 있다. 이 클래스는 **메시지를 주고받으며 서로 대화할 수 있는 에이전트**를 정의한 것으로, **공동으로 작업을 수행할 수 있도록 설계**되어 있다. 에이전트는 다른 에이전트와 통신하며 작업을 수행할 수 있으며, 메시지를 받은 뒤 어떤 행동을 취할지는 에이전트마다 다르게 설정할 수 있다. 대표적인 하위 클래스는 다음 두 가지이다. 

[1] `AssistantAgent` 
- AssistantAgent는 **AI 어시스턴트 역할**을 하도록 설계되었다.
- 기본적으로 LLM을 사용하지만, 사람의 입력이나 코드 실행은 요구하지 않는다.
- 사용자가 어떤 작업을 수행해야 한다는 설명을 포함한 메시지를 보내면, 이 에이전트는 Python 코드 블록을 작성하여 응답할 수 있다.
- 코드 작성은 GPT-4 등의 LLM이 자동으로 수행하며, 실행 결과를 받아 버그 수정이나 보완 제안도 할 수 있다.
- 에이전트의 행동은 새로운 **system message**를 전달함으로써 변경 가능하며, LLM 추론 구성은 `llm_config`를 통해 설정 가능하다.

[2] `UserProxyAgent`
- UserProxyAgent는 **인간 사용자의 대리자 역할**을 하는 개념적 에이전트이다.
- 기본적으로 각 대화 턴마다 사람의 입력을 요청하지만, **코드 실행 및 함수/도구 호출도 수행할 수 있는 기능**을 가지고 있다.
- 메시지에 실행 가능한 코드 블록이 포함되어 있고, 사용자의 일벽이 없는 경우에는 자동으로 코드 실행을 트리거한다.
- 코드 실행을 원하지 않을 경우, `code_execution_config`를  `False`로 설정하면 비활성화할 수 있다.
- LLM 기반의 응답은 기본적으로 비활성화되어 있으나, `llm_config` 에 추론 설정 딕셔너리(dict)을 지정하면 LLM을 이용한 응답이 가능해진다. 
- 이 설정이 적용되면, **코드 실행이 수행되지 않은 경우에는 LLM을 사용해 답변을 생성한다.**

`ConversableAgent`은 자동 응답 기능(auto-reply) 기능을 제공하여, **보다 자율적인 다중 에이전트 커뮤니케이션**을 가능하게 하면서도 **사람의 개입 여지**를 남겨둔다.
또한 `register_reply()` 메서드를 사용하면 **사용자 정의 응답 함수**를 쉽게 등록하여 확장할 수 있다.

다음 코드에서는 *assistant* 라는 이름의 **AssisatntAgent**를 생성하여 어시스턴트 역할을 하도록 하고 *user_proxy*라는 이름의 **UserProxyAgent**를 사용하여 사용자 대리 역할을 하도록 한다.
이후, 두 에이전트를 활용해 하나의 과제를 함께 해결하는 예제를 다룬다.

In [1]:
from config import settings
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import DockerCommandLineCodeExecutor

openAI_api_key = settings.openai_api_key.get_secret_value()
anthropic_api_key = settings.anthropic_api_key.get_secret_value()

In [7]:
config_list = [
    {
        "model" : "gpt-4o-mini",
        "api_key" : openAI_api_key,
    },
    {
        "model" : "claude-3-5-haiku-20241022",
        "api_key": anthropic_api_key,
    }
]


llm_config={
    "config_list" : config_list
}

In [8]:
code_executor = DockerCommandLineCodeExecutor()

code_execution_config={
    "executor": code_executor
}

In [9]:
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)


user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=code_execution_config
)

In [10]:
assistant

In [11]:
user_proxy

## Multi-agent Conversations

### A Basic Two-Agent Conversation Example

참여하는 에이전트들이 제대로 구성되면, 다음 코드에 나와있는 초기화 단계를 통해 다중 에이전트 대화 세션을 시작할 수 있다.


In [12]:
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Which bit texth stock has the largest year-to-date gain this year?
How much is the gain?"""
)

user_proxy (to assistant):

What date is today? Which bit texth stock has the largest year-to-date gain this year?
How much is the gain?

--------------------------------------------------------------------------------
assistant (to user_proxy):

To answer your questions, I will first get the current date. Then I will find the bit text stock with the largest year-to-date gain and provide the gain amount.

### Step 1: Get the current date

I will execute Python code to obtain today's date.

```python
from datetime import datetime

# Get today's date
today = datetime.now().date()
print(today)
```

### Step 2: Find the bit text stock with the largest year-to-date gain

I will retrieve the stock data to determine which stock has the largest year-to-date gain this year. I will use the `yfinance` library to fetch relevant stock data.

To do this, please ensure you have the `yfinance` library installed. You can install it using the command:
```sh
pip install yfinance
```

After installing the

ChatResult(chat_id=None, chat_history=[{'content': 'What date is today? Which bit texth stock has the largest year-to-date gain this year?\nHow much is the gain?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'To answer your questions, I will first get the current date. Then I will find the bit text stock with the largest year-to-date gain and provide the gain amount.\n\n### Step 1: Get the current date\n\nI will execute Python code to obtain today\'s date.\n\n```python\nfrom datetime import datetime\n\n# Get today\'s date\ntoday = datetime.now().date()\nprint(today)\n```\n\n### Step 2: Find the bit text stock with the largest year-to-date gain\n\nI will retrieve the stock data to determine which stock has the largest year-to-date gain this year. I will use the `yfinance` library to fetch relevant stock data.\n\nTo do this, please ensure you have the `yfinance` library installed. You can install it using the command:\n```sh\npip install yfinance\n```\n\nAfter installing the 